In [9]:
import matplotlib.pylab as plt
import pandas as pd
import sqlite3
import json
from functools import partial

In [17]:
db = "./data/haizhu@gz.houses.db.20170106"
con = sqlite3.connect(db)

query = """
SELECT 
"链家编号", "标题", "总价", "建筑面积", "经度", "纬度"
FROM houses;
"""

df = pd.read_sql(query, con)
df["单价"] = df["总价"] / df["建筑面积"]
fn = lambda x, pnt_pos: int(x*pnt_pos+0.5)/pnt_pos
map_fn = partial(fn, pnt_pos=10000.0)
df["经度_集中"] = df["经度"].map(map_fn)
df["纬度_集中"] = df["纬度"].map(map_fn)

In [18]:
api_pattern_file = "./baidu_api_pattern.html"
api_file = "./data/haizhu@gz.houses.html"

df_group = df.groupby([df["经度_集中"], df["纬度_集中"]]).mean()
df_group["单价"] = df_group["单价"].map(partial(fn, pnt_pos=10.0)) * 20.0
to_dict = lambda row : {"lng": row["经度_集中"], "lat": row["纬度_集中"], "count": row["单价"]}
data = [to_dict(df_group.ix[index]) for index in df_group.index]
with open(api_pattern_file) as pattern_f, open(api_file, "w") as api_f:
    pattern = pattern_f.read()
    data_repr = json.dumps(data, ensure_ascii=False)
    data_repr = data_repr.replace("},", "},\n")
    
    center_lnt = (df["经度"].max() + df["经度"].min())/2.0
    center_lat = (df["纬度"].max() + df["纬度"].min())/2.0
    print(center_lat, center_lnt)
    pattern = pattern.replace("{{center_lnt}}", str(center_lnt))
    pattern = pattern.replace("{{center_lat}}", str(center_lat))
    pattern = pattern.replace("{{points}}", data_repr)
    api_f.write(pattern)
df.describe()

23.0856465 113.3249225


,总价,建筑面积,经度,纬度,单价,经度_集中,纬度_集中
count,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000
mean,187.775562,71.560395,113.296068,23.094221,2.455829,113.296070,23.094221
std,162.774340,34.930163,0.023204,0.012685,0.908240,0.023203,0.012683
min,0.000000,6.450000,113.254866,23.052735,0.000000,113.254900,23.052700
25%,100.000000,49.240000,113.280095,23.086034,1.826923,113.280100,23.086000
50%,135.000000,65.560000,113.293860,23.096145,2.126582,113.293900,23.096100
75%,217.000000,82.500000,113.310708,23.104697,2.880000,113.310700,23.104700
max,2354.000000,428.000000,113.394979,23.118558,6.613757,113.395000,23.118600
